In [ ]:
from ccpi.framework import ImageData, ImageGeometry
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.optimisation.algorithms import CGLS, FISTA
from ccpi.optimisation.functions import Norm2Sq, L1Norm
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity
from ccpi.framework import BlockDataContainer

from ccpi.processors import CenterOfRotationFinder

from ccpi.io import NEXUSDataReader
from ccpi.astra.operators import AstraProjector3DSimple

# All external imports
import numpy
import matplotlib.pyplot as plt
import os
import sys
import scipy
from utilities import islicer


In [ ]:
## Set up a reader object pointing to the Nexus data set. Revise path as needed.
# The data is already  corrected for by flat and dark field.

nexus_file=os.path.join(sys.prefix, 'share','ccpi','24737_fd_normalised.nxs')
myreader = NEXUSDataReader(nexus_file=nexus_file)
data_raw = myreader.load_data()
# Negative logarithm transoform.
data_raw.fill( -numpy.log(data_raw.as_array() ))

In [ ]:
print("data shape: ", data_raw.shape)
islicer(data_raw, direction=0)

In [ ]:
from ccpi.processors import Resizer

resizer = Resizer(roi=[-1,(1,135),-1])
resizer.set_input(data_raw)
data_cropped = resizer.get_output()

In [ ]:
data_cropped.shape

In [ ]:
islicer(data_cropped, direction=0)

In [ ]:
# Set up CenterOfRotationFinder object to center data.
# Set the output of the normaliser as the input and execute to determine center.
cor = CenterOfRotationFinder()
cor.set_input(data_cropped)
center_of_rotation = cor.get_output()
print( "centre of rotation at pixel: ", center_of_rotation)

In [ ]:
#exercise use the resizer to crop the image 

In [ ]:
resizer = Resizer(roi=[-1,-1,(12,160)])
resizer.set_input(data_cropped)
data_centred = resizer.get_output()

In [ ]:
islicer(data_centred, direction=0)

In [ ]:
data_centred.shape

In [ ]:
# Permute array and convert angles to radions for ASTRA
data = data_centred.subset(dimensions=['vertical','angle','horizontal'])
data.geometry = data_centred.geometry
data.geometry.angles = data_centred.geometry.angles*numpy.pi/180.

In [ ]:
islicer(data, direction=1)

In [ ]:
# Create Acquisition and Image Geometries for setting up projector.
ag = data.geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_num_y=ag.pixel_num_h, 
                   voxel_num_z=ag.pixel_num_v)
ag.dimension_labels = ['vertical','angle','horizontal']
# Define the projector object
print ("Define projector")
Cop = AstraProjector3DSimple(ig, ag)

In [ ]:
# Test backprojection and projection
z1 = Cop.adjoint(data)
z2 = Cop.direct(z1)

plt.imshow(z1.subset(horizontal_x=68).array)
plt.show()

In [ ]:
#define the operator A
L = Gradient(ig)
alpha = 0
operator_block = BlockOperator( Cop, alpha * L, shape=(2,1))

In [ ]:
#define the data b
data_block = BlockDataContainer(data, L.range_geometry().allocate())

In [ ]:
#setup CGLS
x_init = ig.allocate(0)
cgls = CGLS(x_init=x_init, operator=operator_block, data=data_block)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

In [ ]:
#run the algorithm
cgls.run(100, verbose = True)

In [ ]:
x_CGLS_0 = cgls.get_output()
x_CGLS_0_objective = cgls.objective

In [ ]:
x_CGLS_reg = cgls.get_output()
x_CGLS_reg_objective = cgls.objective

In [ ]:
#output = x_CGLS.as_array()
#f=open("testout.raw","wb")
#output.astype(dtype=numpy.float32).tofile(f)

In [ ]:
#print(output.shape)

In [ ]:
islicer(x_CGLS_0, direction=0)
islicer(x_CGLS_reg, direction=0)

In [ ]:

x_CGLS = x_CGLS_reg
x_CGLS_objective = x_CGLS_reg_objective

In [ ]:
# Fix color and slices for display
v1 = -0.01
v2 = 0.13
hx=80
hy=80
v=68

# Display ortho slices of reconstruction
# Display all reconstructions and decay of objective function
cols = 3
rows = 1
fig = plt.figure()

current = 1
a=fig.add_subplot(rows,cols,current)
a.set_title('horizontal_x')
imgplot = plt.imshow(x_CGLS.subset(horizontal_x=hx).as_array(),vmin=v1,vmax=v2)

current = current + 1
a=fig.add_subplot(rows,cols,current)
a.set_title('horizontal_y')
imgplot = plt.imshow(x_CGLS.subset(horizontal_y=hy).as_array(),vmin=v1,vmax=v2)

current = current + 1
a=fig.add_subplot(rows,cols,current)
a.set_title('vertical')
imgplot = plt.imshow(x_CGLS.subset(vertical=v).as_array(),vmin=v1,vmax=v2)
plt.colorbar()

plt.suptitle('CGLS reconstruction slices')
plt.show()

plt.figure()
plt.semilogy(x_CGLS_objective)
plt.title('CGLS criterion')
plt.show()

In [ ]:
utilities.interactive_slice_display(x_CGLS, direction=0)